# Imports

In [1]:
# importing required libraries
import pandas as pd
import numpy as np
import category_encoders as ce
import matplotlib.pyplot as plt
import re
from datetime import datetime as dt
import calendar as cl
from scipy.interpolate import interp1d

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Read Data

In [2]:
# Read in data
temp = pd.read_csv('/Users/mfairb/Documents/ML Projects/Project - Pythonic Time Series/Milestone 1 Dataset/hr_temp_20170201-20200131_subset.csv')
#temp2 = pd.read_csv('/Users/mfairb/Documents/ML Projects/Project - Pythonic Time Series/Milestone 1 Dataset/hr_temp_20200201-20200229_subset.csv')
meter = pd.read_csv('/Users/mfairb/Documents/ML Projects/Project - Pythonic Time Series/Milestone 1 Dataset/hrl_load_metered - 20170201-20200131.csv')

# Cleaning

In [4]:
# Keep only needed columns in temp and meter
temp = temp[['DATE', 'HourlyDryBulbTemperature']]
meter = meter[['datetime_beginning_utc', 'datetime_beginning_ept', 'mw']]

# Adjust column names
temp = temp.rename(columns={'DATE':'date'})
meter = meter.rename(columns={'datetime_beginning_utc':'b_utc',
                              'datetime_beginning_ept':'b_ept'})

# Standardize time format between sets (using format from temp)
temp['date'] = pd.to_datetime(temp['date'])
meter['b_utc'] = pd.to_datetime(meter['b_utc'])
meter['b_ept'] = pd.to_datetime(meter['b_ept'])

# Adjust time marks for dates (make equal to nearest hour)
        # ex: 2020-01-12 03:53:00 -> 2020-01-12 04:00:00
        # ex: 2019-12-31 23:53:00 -> 2020-01-00 00:00:00
def closest_date(date_set, date):
    if date.hour==23:
        return min(date_set, key=lambda x: abs(x - date))
    else:
        return date.replace(hour = date.hour + 1,
                            minute = 0)

temp['date'] = [closest_date(meter.b_ept, t) for t in temp.date]

# Creating Train vs. Test

In [6]:
# Join on date (date & b_ept)
df = pd.merge(temp, meter, left_on='date', right_on='b_ept', how='inner')

# Create hour, day, month, year columns
new_columns = ['hour', 'day', 'month', 'year']
for c in new_columns:
    df[c] = [eval(f't.{c}') for t in df.date]

# Adjust columns and set index
df = df.rename(columns={'HourlyDryBulbTemperature':'temp'})
df = df[['date','mw','temp','hour','day', 'month', 'year']]
df = df.set_index('date')

# Interpolating
df['key'] = range(0, df.shape[0])
f = interp1d(df.key[~df.temp.isnull()], df.temp[~df.temp.isnull()], kind='linear')
df['temp'] = f(df.key)
df = df.drop(['key'], axis=1)

# Create train & test
train = df[df.year < 2020]
test = df[df.year >= 2020]